# PROYECTO FINAL - APACHE SPARK

## Caso:

La pizzeria Danny quiere usar los datos para responder algunas preguntas simples sobre sus
clientes, especialmente sobre sus patrones de visitas, cuánto dinero han gastado y también qué
elementos del menú son sus favoritos.

![](diagram.png)

* Preguntas a resolver usando Apache Spark:
    
    1. ¿Cuál es la cantidad total que gastó cada cliente en el restaurante?

    2. ¿Cuántos días ha visitado cada cliente el restaurante?
    
    3. ¿Cuál fue el primer artículo del menú comprado por cada cliente?
    
    4. ¿Cuál es el artículo más comprado en el menú y cuántas veces lo compraron todos los
    clientes?

    5. ¿Qué artículo fue el más popular para cada cliente?

    6. ¿Qué artículo compró primero el cliente después de convertirse en miembro?

    7. ¿Qué artículo se compró justo antes de que el cliente se convirtiera en miembro?
    
    8. ¿Cuál es el total de artículos y la cantidad gastada por cada miembro antes de
    convertirse en miembro?
    
    9. Si cada $ 1 gastado equivale a 10 puntos y el sushi tiene un multiplicador de puntos 2x, ¿cuántos puntos tendría cada cliente?
    
    10. En la primera semana después de que un cliente se une al programa (incluida la fecha
    de ingreso), gana el doble de puntos en todos los artículos, no solo en sushi. ¿Cuántos
    puntos tienen los clientes A y B a fines de enero?

In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkFiles
spark = SparkSession.builder.\
                    master("local[3]").\
                    config('spark.jars', 'gs://spark-lib/bigquery/spark-bigquery-with-dependencies_2.12-0.23.1').\
                    getOrCreate()
                    
                    

spark

In [3]:
url_members = "https://raw.githubusercontent.com/datahackformation/DEP7_Apache-Spark/main/Trabajo%20Final/Data/members.csv"
spark.sparkContext.addFile(url_members)
members = spark.read.option("header", True).csv(SparkFiles.get("members.csv"),inferSchema=True)
members.show()

+-----------+----------+
|customer_id| join_date|
+-----------+----------+
|          A|2021-01-07|
|          B|2021-01-09|
+-----------+----------+



In [4]:
members.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- join_date: string (nullable = true)



In [5]:
url_menu = "https://raw.githubusercontent.com/datahackformation/DEP7_Apache-Spark/main/Trabajo%20Final/Data/menu.csv"
spark.sparkContext.addFile(url_menu)
menu = spark.read.option("header", True).csv(SparkFiles.get("menu.csv"),inferSchema=True)
menu.show()

+----------+------------+-----+
|product_id|product_name|price|
+----------+------------+-----+
|         1|       sushi|   10|
|         2|       curry|   15|
|         3|       ramen|   12|
+----------+------------+-----+



In [6]:
menu.printSchema()

root
 |-- product_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- price: integer (nullable = true)



In [7]:
url_sales = "https://raw.githubusercontent.com/datahackformation/DEP7_Apache-Spark/main/Trabajo%20Final/Data/sales.csv"
spark.sparkContext.addFile(url_sales)
sales = spark.read.option("header", True).csv(SparkFiles.get("sales.csv"),inferSchema=True)
sales.show()

+-----------+----------+----------+
|customer_id|order_date|product_id|
+-----------+----------+----------+
|          A|2021-01-01|         1|
|          A|2021-01-01|         2|
|          A|2021-01-07|         2|
|          A|2021-01-10|         3|
|          A|2021-01-11|         3|
|          A|2021-01-11|         3|
|          B|2021-01-01|         2|
|          B|2021-01-02|         2|
|          B|2021-01-04|         1|
|          B|2021-01-11|         1|
|          B|2021-01-16|         3|
|          B|2021-02-01|         3|
|          C|2021-01-01|         3|
|          C|2021-01-01|         3|
|          C|2021-01-07|         3|
+-----------+----------+----------+



In [8]:
sales.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- order_date: string (nullable = true)
 |-- product_id: integer (nullable = true)



### Pregunta 1: 

¿Cuál es la cantidad total que gastó cada cliente en el restaurante?

In [9]:
sales.join(menu,sales.product_id ==  menu.product_id,"inner") \
     .select('customer_id','price')\
     .groupBy('customer_id').sum('price')\
     .orderBy('customer_id')\
     .show(truncate=False)


+-----------+----------+
|customer_id|sum(price)|
+-----------+----------+
|A          |76        |
|B          |74        |
|C          |36        |
+-----------+----------+



### Pregunta 2:
 
¿Cuántos días ha visitado cada cliente el restaurante?

In [10]:
from pyspark.sql.functions import countDistinct

sales.select('customer_id','order_date')\
     .groupBy('customer_id').agg(countDistinct('order_date'))\
     .orderBy('customer_id')\
     .show(truncate=False)

+-----------+-----------------+
|customer_id|count(order_date)|
+-----------+-----------------+
|A          |4                |
|B          |6                |
|C          |2                |
+-----------+-----------------+



### Pregunta 3:
 
¿Cuál fue el primer artículo del menú comprado por cada cliente?

In [14]:
# Usando funciones spark
from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number

w2 = Window.partitionBy("customer_id").orderBy(col('order_date'))
sales.withColumn("row",row_number().over(w2))\
  .filter(col("row") == 1).drop("row")\
  .join(menu,sales.product_id ==  menu.product_id,"inner")\
  .select('customer_id','order_date','product_name')\
  .show()


+-----------+----------+------------+
|customer_id|order_date|product_name|
+-----------+----------+------------+
|          A|2021-01-01|       sushi|
|          B|2021-01-01|       curry|
|          C|2021-01-01|       ramen|
+-----------+----------+------------+



In [15]:
# Usando SQL query
sales.createOrReplaceTempView("Sales")
menu.createOrReplaceTempView("Menu")
spark.sql("select customer_id, order_date, product_name from "+
     " (select *, row_number() OVER (PARTITION BY customer_id ORDER BY order_date) as rn " +
     " FROM Sales) tmp inner join Menu as M on tmp.product_id = M.product_id where rn = 1").show()

+-----------+----------+------------+
|customer_id|order_date|product_name|
+-----------+----------+------------+
|          A|2021-01-01|       sushi|
|          B|2021-01-01|       curry|
|          C|2021-01-01|       ramen|
+-----------+----------+------------+



### Pregunta 4:
 
¿Cuál es el artículo más comprado en el menú y cuántas veces lo compraron todos los clientes?

In [26]:
# Usando funciones Spark
import pyspark.sql.functions as f

w = Window.partitionBy('customer_id')
sales.groupBy('customer_id','product_id').count()\
     .withColumn('maxCount', f.max('count').over(w))\
     .where(f.col('count') == f.col('maxCount')).drop('maxCount')\
     .join(menu,sales.product_id == menu.product_id)\
     .select('customer_id','product_name','count')\
     .show()

+-----------+------------+-----+
|customer_id|product_name|count|
+-----------+------------+-----+
|          A|       ramen|    3|
|          B|       sushi|    2|
|          B|       ramen|    2|
|          B|       curry|    2|
|          C|       ramen|    3|
+-----------+------------+-----+



In [42]:
sales.createOrReplaceTempView("Sales")
menu.createOrReplaceTempView("Menu")
spark.sql("select customer_id, product_name, count from " +
          "(select *, MAX(count) OVER (PARTITION BY customer_id) AS maxCount from " +
          "(select customer_id, product_id, count(product_id) as count from Sales group by customer_id, product_id) tmp) tf " +
          "inner join Menu as M on tf.product_id = M.product_id where maxCount = count").show()

+-----------+------------+-----+
|customer_id|product_name|count|
+-----------+------------+-----+
|          A|       ramen|    3|
|          B|       sushi|    2|
|          B|       ramen|    2|
|          B|       curry|    2|
|          C|       ramen|    3|
+-----------+------------+-----+



### Pregunta 5:
 
¿Qué artículo fue el más popular para cada cliente?

In [43]:
import pyspark.sql.functions as f

w = Window.partitionBy('customer_id')
sales.groupBy('customer_id','product_id').count()\
     .withColumn('maxCount', f.max('count').over(w))\
     .where(f.col('count') == f.col('maxCount')).drop('maxCount')\
     .join(menu,sales.product_id == menu.product_id)\
     .select('customer_id','product_name')\
     .show()

+-----------+------------+
|customer_id|product_name|
+-----------+------------+
|          A|       ramen|
|          B|       sushi|
|          B|       ramen|
|          B|       curry|
|          C|       ramen|
+-----------+------------+



In [59]:
sales.createOrReplaceTempView("Sales")
menu.createOrReplaceTempView("Menu")
spark.sql("select customer_id, product_name from" +
          "(select customer_id, product_id, count, MAX(count) OVER (PARTITION BY customer_id) AS maxCount from" +
          "(select customer_id, product_id, count(product_id) as count from Sales group by customer_id, product_id) tmp) tf " +
          "inner join Menu as m on tf.product_id = m.product_id where maxCount = count").show()

+-----------+------------+
|customer_id|product_name|
+-----------+------------+
|          A|       ramen|
|          B|       sushi|
|          B|       ramen|
|          B|       curry|
|          C|       ramen|
+-----------+------------+



### Pregunta 6:
 
¿Qué artículo compró primero el cliente después de convertirse en miembro?


In [73]:
w6 = Window.partitionBy("customer_id").orderBy(col('order_date'))
sales.join(members, ["customer_id"],"inner")\
     .where(col("order_date")>=col("join_date"))\
     .withColumn("row",row_number().over(w6))\
     .filter(col("row")==1).drop("row")\
     .show(truncate=False)

+-----------+----------+----------+----------+
|customer_id|order_date|product_id|join_date |
+-----------+----------+----------+----------+
|A          |2021-01-07|2         |2021-01-07|
|B          |2021-01-11|1         |2021-01-09|
+-----------+----------+----------+----------+



In [84]:
sales.createOrReplaceTempView("Sales")
members.createOrReplaceTempView("Members")
spark.sql("select customer_id, order_date, product_id, join_date "+
          "from (select *, row_number() OVER (PARTITION BY customer_id ORDER BY order_date) as rn from "+
          "(select s.customer_id as customer_id, order_date, product_id, join_date from Sales s "+ 
          "inner join Members m on s.customer_id = m.customer_id where order_date >= join_date) temp) tf "+
          "where rn = 1").show()

+-----------+----------+----------+----------+
|customer_id|order_date|product_id| join_date|
+-----------+----------+----------+----------+
|          A|2021-01-07|         2|2021-01-07|
|          B|2021-01-11|         1|2021-01-09|
+-----------+----------+----------+----------+



### Pregunta 7:

¿Qué artículo se compró justo antes de que el cliente se convirtiera en miembro?

In [96]:
w6 = Window.partitionBy("customer_id").orderBy(col('order_date').desc())
sales.join(members, ["customer_id"],"inner")\
     .where(col("order_date")<col("join_date"))\
     .withColumn("row",row_number().over(w6))\
     .filter(col("row")==1).drop("row")\
     .show(truncate=False)

+-----------+----------+----------+----------+
|customer_id|order_date|product_id|join_date |
+-----------+----------+----------+----------+
|A          |2021-01-01|1         |2021-01-07|
|B          |2021-01-04|1         |2021-01-09|
+-----------+----------+----------+----------+



In [95]:
sales.createOrReplaceTempView("Sales")
members.createOrReplaceTempView("Members")
spark.sql("select customer_id, order_date, product_id, join_date "+
          "from (select *, row_number() OVER (PARTITION BY customer_id ORDER BY order_date desc) as rn from "+
          "(select s.customer_id as customer_id, order_date, product_id, join_date from Sales s "+ 
          "inner join Members m on s.customer_id = m.customer_id where order_date < join_date) temp) tf "+
          "where rn = 1").show()

+-----------+----------+----------+----------+
|customer_id|order_date|product_id| join_date|
+-----------+----------+----------+----------+
|          A|2021-01-01|         1|2021-01-07|
|          B|2021-01-04|         1|2021-01-09|
+-----------+----------+----------+----------+



### Pregunta 8:

¿Cuál es el total de artículos y la cantidad gastada por cada miembro antes de convertirse en miembro?

In [101]:
sales.join(members, ["customer_id"],"inner")\
     .where(col("order_date")<col("join_date"))\
     .join(menu, ["product_id"],"inner")\
     .select("customer_id","order_date","product_id","price")\
     .groupBy("customer_id").agg({"order_date":"count","price":"sum"})\
     .show()

+-----------+-----------------+----------+
|customer_id|count(order_date)|sum(price)|
+-----------+-----------------+----------+
|          B|                3|        40|
|          A|                2|        25|
+-----------+-----------------+----------+



In [109]:
sales.createOrReplaceTempView("Sales")
members.createOrReplaceTempView("Members")
menu.createOrReplaceTempView("Menu")

spark.sql("select customer_id, count(order_date) as countSales, sum(price) as Total from "+
          "(select s.customer_id, order_date, s.product_id, price from Sales s "+
          "inner join Members m on s.customer_id = m.customer_id "+
          "inner join Menu me on s.product_id = me.product_id "+
          "where order_date < join_date) temp group by customer_id").show()

+-----------+----------+-----+
|customer_id|countSales|Total|
+-----------+----------+-----+
|          B|         3|   40|
|          A|         2|   25|
+-----------+----------+-----+



### Pregunta 9:

Si cada $ 1 gastado equivale a 10 puntos y el sushi tiene un multiplicador de puntos 2x, ¿cuántos puntos tendría cada cliente?

In [115]:
sales.join(menu,["product_id"]).withColumn("mult",col("product_name")=="sushi")\
     .withColumn("mult",col("mult").cast("int"))\
     .withColumn("points",col("Price")*10 + col("mult")*(col("Price")*10))\
     .groupBy("customer_id").sum("points")\
     .show()

+-----------+-----------+
|customer_id|sum(points)|
+-----------+-----------+
|          B|        940|
|          C|        360|
|          A|        860|
+-----------+-----------+



In [122]:
sales.createOrReplaceTempView("Sales")
menu.createOrReplaceTempView("Menu")

spark.sql("select customer_id, sum(points) as total_points from "+
          "(select customer_id,price*10 + mult*price*10 as points from "+
          "(select *,case when m.product_name = 'sushi' then 1 else 0 end as mult from "+
          "Sales s inner join Menu m on s.product_id = m.product_id) temp) tf "+
          "group by customer_id").show()

+-----------+------------+
|customer_id|total_points|
+-----------+------------+
|          B|         940|
|          C|         360|
|          A|         860|
+-----------+------------+



### Pregunta 10

En la primera semana después de que un cliente se une al programa (incluida la fecha
de ingreso), gana el doble de puntos en todos los artículos, no solo en sushi. ¿Cuántos
puntos tienen los clientes A y B a fines de enero?

In [133]:
from pyspark.sql.functions import when

sales.join(menu,["product_id"],"inner")\
     .join(members,["customer_id"],"inner")\
     .withColumn("mult_member",when(col("order_date")>=col("join_date"),1).when(col("product_name")=="sushi",1).otherwise(0))\
     .withColumn("mult_member",col("mult_member").cast("int"))\
     .withColumn("points",col("Price")*10 + col("mult_member")*(col("Price")*10))\
     .filter(col("order_date")<'2021-01-31')\
     .groupBy("customer_id").sum("points")\
     .show()

+-----------+-----------+
|customer_id|sum(points)|
+-----------+-----------+
|          B|        940|
|          A|       1370|
+-----------+-----------+



In [145]:
sales.createOrReplaceTempView("Sales")
members.createOrReplaceTempView("Members")
menu.createOrReplaceTempView("Menu")

spark.sql("select customer_id, sum(points) as total_points from "+
          "(select customer_id,order_date, price*10 + mult_member*price*10 as points from "+
          "(select *, case when order_date>=join_date then 1 when product_name='sushi' then 1 else 0 end as mult_member from "+
          "(select s.customer_id,s.order_date,m.product_name,m.price,me.join_date from Sales s "+
          "inner join Menu m on s.product_id = m.product_id "+
          "inner join Members me on s.customer_id = me.customer_id) temp) temp2 "+
          "where order_date<'2021-01-31') tf group by customer_id").show()

+-----------+------------+
|customer_id|total_points|
+-----------+------------+
|          B|         940|
|          A|        1370|
+-----------+------------+

